In [11]:
import numpy as np
from qwanta import Experiment
import matplotlib.pyplot as plt
import pandas as pd
from math import radians, cos, sin, asin, sqrt
import dill
import requests
import ray

ray.init()

{'node_ip_address': '127.0.0.1',
 'raylet_ip_address': '127.0.0.1',
 'redis_address': None,
 'object_store_address': 'tcp://127.0.0.1:63205',
 'raylet_socket_name': 'tcp://127.0.0.1:64976',
 'webui_url': None,
 'session_dir': 'C:\\Users\\User\\AppData\\Local\\Temp\\ray\\session_2022-04-07_10-05-39_383702_27784',
 'metrics_export_port': 56375,
 'gcs_address': '127.0.0.1:57297',
 'address': '127.0.0.1:57297',
 'node_id': 'd1ed03560b36fd022b0ad370a09a6d532415594e452cd94fd35cbef5'}

# Define function in this simulation

In [12]:
# Memory error
def memError(time, tau=1):
    p = (np.e**(-1*(time/tau)))/4 + 0.75
    return [p, (1- p)/3, (1- p)/3, (1- p)/3]

def ISS_orbit(t):
    # [5.71756095e+01 1.06960339e+00 8.52707152e-01 4.88544727e-04]
    
    lat_t = 57.18*sin(radians(1.07*t + 8.53)) + 4.89
    lon_t = 3.63*t - 22.58
    r_iss = 6771
    x = r_iss*np.cos(lat_t)*np.cos(lon_t)
    y = r_iss*np.cos(lat_t)*np.sin(lon_t)
    z = r_iss*np.sin(lat_t)

    return (x, y, z)

def earth_location(lat, lon):
    lat, lon = np.deg2rad(lat), np.deg2rad(lon)
    r_e = 6371
    x = r_e*np.cos(lat)*np.cos(lon)
    y = r_e*np.cos(lat)*np.sin(lon)
    z = r_e*np.sin(lat)
    return (x, y, z)

def place(choose):
    if choose == 'BKK':
        return earth_location(13.4553, 100.31316)
    elif choose == 'CM':
        return earth_location(18.804982, 98.954428)
    elif choose == 'SK':
        return earth_location(7.213724, 100.594990)
    elif choose == 'Keio':
        return earth_location(35.38563, 139.44346)

def bot(message, Disnotification=False):
    token = 'g7AsCP08s0SQvcNQDQFF53lfizJTB6yYr8s27PVA7mc'
    payload = {'message' : message
          ,'notificationDisabled' : Disnotification
          , 'stickerPackageId' : '789'
          , 'stickerId' : '10857'}
    r = requests.post('https://notify-api.line.me/api/notify'
                , headers={'Authorization' : 'Bearer {}'.format(token)}
                , params = payload)


# Create experiment and run

In [13]:
loss = [0.012] # it can setting to vary loss
p_dep = [0.97, 0.01, 0.01, 0.01] # independent with distant
gateErr =  [0.9975, 0.0025]
meaErr = 0.01
pickle_file_name = ['BKK_CM_0.012_tau1_meaErr1']
exp_names = ['EPPS']

# Please do not forget to check the place before start runing program
nodes_info_exp = [{
            'Node 0': {'coordinate': place('BKK')},
            'Node 1': {'coordinate': ISS_orbit},
            'Node 2': {'coordinate': place('CM')},
            'numPhysicalBuffer': 20,
            'numInternalEncodingBuffer': 20,
            'numInternalDetectingBuffer': 10,
            'numInternalInterfaceBuffer': 2,
        },
        ]

# experiment base on loss

@ray.remote
def loss_vary(l, j):
    Quantum_topology = [{
            ('Node 0', 'Node 1'): {
            'connection-type': 'Space',
            'function': p_dep,
            'loss': l,
            'light speed': 300000, # km/s
            'Pulse rate': 0.0001, # waiting time for next qubit (interval)
            },
            ('Node 1', 'Node 2'): {
            'connection-type': 'Space',
            'function': p_dep,
            'loss': l,
            'light speed': 300000,
            'Pulse rate': 0.0001,
            },
            ('Node 0', 'Node 2'): {
            'connection-type': 'Space',
            'function': p_dep,
            'loss': l,
            'light speed': 300000,
            'Pulse rate': 0.0001,
            },
        }
    for _ in exp_names]

# for i in range(len(loss)):
#     Quantum_topology = [{
#             ('Node 0', 'Node 1'): {
#             'connection-type': 'Space',
#             'function': p_dep,
#             'loss': loss[i],
#             'light speed': 300000, # km/s
#             'Pulse rate': 0.0001, # waiting time for next qubit (interval)
#             },
#             ('Node 1', 'Node 2'): {
#             'connection-type': 'Space',
#             'function': p_dep,
#             'loss': loss[i],
#             'light speed': 300000,
#             'Pulse rate': 0.0001,
#             },
#             ('Node 0', 'Node 2'): {
#             'connection-type': 'Space',
#             'function': p_dep,
#             'loss': loss[i],
#             'light speed': 300000,
#             'Pulse rate': 0.0001,
#             },
#         }
#     for _ in exp_names]



    timelines = {}
    for exp_name in exp_names:
        e_tl, vis_a = Experiment.read_timeline_from_csv(f'experssdp.xlsx', excel=True, sheet_name=exp_name) 
        timelines[exp_name] = e_tl
        
    e_tl[2]['Resource Type'] = 'Physical'
    e_tl[2]['Edges'] = ['Node 0', 'Node 2']
    e_tl[2]['Num Trials'] = 9000

    nodes_information = {exp_name: nodes_info_exp[index] for index, exp_name in enumerate(exp_names)}
    networks = {exp_name: Quantum_topology[index] for index, exp_name in enumerate(exp_names)}
    mem_func = {exp_name: memError for exp_name in exp_names}
    gate_error = {exp_name: gateErr for exp_name in exp_names}
    measure_error = {exp_name: meaErr for exp_name in exp_names}
    sim_time = {exp_name: None for exp_name in exp_names}
    labels = {exp_name: 'Physical' for exp_name in exp_names}

    p = [0]
    exper = Experiment(networks, timelines, measurementError=measure_error, nodes_info=nodes_information, memFunc=mem_func, gateError=gate_error, simTime=sim_time,
                    parameters_set=p, collect_fidelity_history=True, repeat=5, 
                    label_records=labels,path=j, message_log='epps', progress_bar=True)

    exper.run()

result = ray.get([loss_vary.remote(i, j) for i, j in zip(loss, pickle_file_name)])

 pid=23168) Parameters executed:   0%|          | 0/1 [00:00<?, ?it/s]


In [ ]:
message = 'Your remote first simulation is done'
bot(message)

In [ ]:
ray.shutdown()